# Base model with Bidirectional Encoder Representations from Transformers (BERT)

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import missingno as msno
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from keras.preprocessing import text
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer,BertTokenizerFast, BertForSequenceClassification
from transformers import TrainingArguments, Trainer, pipeline


2024-10-23 15:15:24.474018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prepare Dataset

Load JSON file

In [2]:
def load_json_file(filename):
    with open(filename) as f:
        file=json.load(f)
    return file   
    
filename="intents.json"
intents=load_json_file(filename)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hey there', 'hello there'], 'responses': ['Hello How may I help You?', 'Hi,How can I help You?', 'Hi there,What Can I do for You Today?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again'], 'context_set': ''}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"], 'context_set': ''}, {'tag': 'about', 'patterns': ['Who are you?', 'What are you?', 'Who you are?'], 'responses': ["I'm MMCOE's intelligent bot assistant, here to assist you!"], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['A bot has no name. But I have a purpose, and that is to help you. How can I help you?'], 'context_s

Create dataframe for patterns and tags

In [3]:
def create_df():
    df=pd.DataFrame({'Pattern':[],'Tag':[]})
    return df
    
df=create_df()
df

,Pattern,Tag


Extract data from file

In [4]:
def extract_json_info(json_file,df):
    for intent in json_file['intents']:
        for pattern in intent['patterns']:
            sentence_tag=[pattern,intent['tag']]
            df.loc[len(df.index)] = sentence_tag
    return df        
df=extract_json_info(intents,df)

## Explore data

In [5]:
df.head()

,Pattern,Tag
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Hey there,greeting


In [6]:
df.shape

(914, 2)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 914 entries, 0 to 913
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Pattern  914 non-null    object
 1   Tag      914 non-null    object
dtypes: object(2)
memory usage: 21.4+ KB


In [8]:
x=len(df['Tag'].unique())
print(f"{x} classes in or tags")

print(df['Tag'].unique())

134 classes in or tags
['greeting' 'goodbye' 'thanks' 'about' 'name' 'complaint' 'contact'
 'social connect' 'working-hours' 'location' 'why-choose-mmcoe'
 'what-we-offer' 'admission-enquiry' 'admission-after-allotment'
 'admission_in_charge_FE' 'admission_in_charge_DSE'
 'admission_in_charge_ME_MBA' 'Computer-Intake' 'IT-Intake'
 'Mechanical-Intake' 'ENTC-Intake' 'Electrical-Intake'
 'DSE-computer-Intake' 'DSE-IT-Intake' 'DSE-Mechanical-Intake'
 'DSE-ENTC-Intake' 'DSE-Electrical-Intake' 'ME-Intake' 'MBA-Intake'
 'FE-Eligibility' 'DSE-Eligibility' 'ME-Eligibility' 'MBA-Eligibility'
 'Documents-FE' 'fee-payment-process' 'Fe-FEES' 'DSE-FEES' 'MBA-FEES'
 'ME-FEES' 'scholarships' 'Faculty-College-Computer' 'Faculty-College-IT'
 'Faculty-College-Mechanical' 'Faculty-College-E&TC'
 'Faculty-College-Electrical' 'Faculty-College-MBA'
 'Faculty-College-Engineering-Sciences' 'Research-Projects'
 'Student-projects' 'About-MMCOE' 'About-Marathwada Mitramandal'
 'About In general' "Principal's mess

In [9]:
df.isna().sum()

Pattern    0
Tag        0
dtype: int64

Understand the corpus

In [10]:
def get_corpus(series):
    words = []
    for text in series:
        for word in text.split():
            words.append(word.strip())
    return words

corpus = get_corpus(df.Pattern)
corpus[:5]
# print(corpus)

['Hi', 'Hey', 'Is', 'anyone', 'there?']

In [11]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
print(most_common)
most_common = dict(most_common)
most_common

[('the', 449), ('for', 416), ('is', 308), ('What', 287), ('of', 183), ('are', 140), ('what', 128), ('cutoff', 119), ('admission', 112), ('department', 92)]


{'the': 449,
 'for': 416,
 'is': 308,
 'What': 287,
 'of': 183,
 'are': 140,
 'what': 128,
 'cutoff': 119,
 'admission': 112,
 'department': 92}

Create another dataframe for patterns, tags and labels

In [13]:
df2 = df.copy()
df2.head()

,Pattern,Tag
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Hey there,greeting


Create list of unique tags as labels

In [14]:
labels=df2['Tag'].unique().tolist()
labels=[s.strip() for s in labels]
labels[:5]

['greeting', 'goodbye', 'thanks', 'about', 'name']

Create `id` to `label` and vise versa

In [15]:
id2label={id:label for id,label in enumerate(labels)}
label2id={label:id for id,label in enumerate(labels)}
num_labels=len(labels)

In [16]:
id2label

{0: 'greeting',
 1: 'goodbye',
 2: 'thanks',
 3: 'about',
 4: 'name',
 5: 'complaint',
 6: 'contact',
 7: 'social connect',
 8: 'working-hours',
 9: 'location',
 10: 'why-choose-mmcoe',
 11: 'what-we-offer',
 12: 'admission-enquiry',
 13: 'admission-after-allotment',
 14: 'admission_in_charge_FE',
 15: 'admission_in_charge_DSE',
 16: 'admission_in_charge_ME_MBA',
 17: 'Computer-Intake',
 18: 'IT-Intake',
 19: 'Mechanical-Intake',
 20: 'ENTC-Intake',
 21: 'Electrical-Intake',
 22: 'DSE-computer-Intake',
 23: 'DSE-IT-Intake',
 24: 'DSE-Mechanical-Intake',
 25: 'DSE-ENTC-Intake',
 26: 'DSE-Electrical-Intake',
 27: 'ME-Intake',
 28: 'MBA-Intake',
 29: 'FE-Eligibility',
 30: 'DSE-Eligibility',
 31: 'ME-Eligibility',
 32: 'MBA-Eligibility',
 33: 'Documents-FE',
 34: 'fee-payment-process',
 35: 'Fe-FEES',
 36: 'DSE-FEES',
 37: 'MBA-FEES',
 38: 'ME-FEES',
 39: 'scholarships',
 40: 'Faculty-College-Computer',
 41: 'Faculty-College-IT',
 42: 'Faculty-College-Mechanical',
 43: 'Faculty-College-E&

In [17]:
label2id

{'greeting': 0,
 'goodbye': 1,
 'thanks': 2,
 'about': 3,
 'name': 4,
 'complaint': 5,
 'contact': 6,
 'social connect': 7,
 'working-hours': 8,
 'location': 9,
 'why-choose-mmcoe': 10,
 'what-we-offer': 11,
 'admission-enquiry': 12,
 'admission-after-allotment': 13,
 'admission_in_charge_FE': 14,
 'admission_in_charge_DSE': 15,
 'admission_in_charge_ME_MBA': 16,
 'Computer-Intake': 17,
 'IT-Intake': 18,
 'Mechanical-Intake': 19,
 'ENTC-Intake': 20,
 'Electrical-Intake': 21,
 'DSE-computer-Intake': 22,
 'DSE-IT-Intake': 23,
 'DSE-Mechanical-Intake': 24,
 'DSE-ENTC-Intake': 25,
 'DSE-Electrical-Intake': 26,
 'ME-Intake': 27,
 'MBA-Intake': 28,
 'FE-Eligibility': 29,
 'DSE-Eligibility': 30,
 'ME-Eligibility': 31,
 'MBA-Eligibility': 32,
 'Documents-FE': 33,
 'fee-payment-process': 34,
 'Fe-FEES': 35,
 'DSE-FEES': 36,
 'MBA-FEES': 37,
 'ME-FEES': 38,
 'scholarships': 39,
 'Faculty-College-Computer': 40,
 'Faculty-College-IT': 41,
 'Faculty-College-Mechanical': 42,
 'Faculty-College-E&TC':

Add converted `label2id` to df2

In [18]:
df2['labels'] = df2['Tag'].map(lambda x: label2id[x.strip()])
df2.head()

,Pattern,Tag,labels
0,Hi,greeting,0
1,Hey,greeting,0
2,Is anyone there?,greeting,0
3,Hello,greeting,0
4,Hey there,greeting,0


In [19]:
X=list(df2['Pattern'])
X[:5]

['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hey there']

In [20]:
y=list(df2['labels'])
y[:5]

[0, 0, 0, 0, 0]

Define training and test datasets

In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=100)

Define base model

In [22]:
model_name="bert-base-uncased"
max_len=256

Load tokenizer

In [23]:
tokenizer=BertTokenizer.from_pretrained(model_name,max_length=max_len)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Load model

In [24]:
model=BertForSequenceClassification.from_pretrained(model_name,num_labels=num_labels,id2label=id2label,label2id=label2id)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Define encoders

In [25]:
train_encoding=tokenizer(X_train,truncation=True,padding=True)
test_encoding=tokenizer(X_test,truncation=True,padding=True)

In [26]:
full_data = tokenizer(X, truncation=True, padding=True)

Data loader

In [27]:
class DataLoader(Dataset):
    
    def __init__(self, encodings, labels):
        
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
               
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):

        return len(self.labels)

In [28]:
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)

In [29]:
fullDataLoader = DataLoader(full_data, y_test)

In [30]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids  # True labels
    preds = pred.predictions.argmax(-1)  # Predicted labels (argmax along the last axis)

    # Using precision_recall_fscore_support to compute precision, recall, F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')

    # Using accuracy_score to compute accuracy
    acc = accuracy_score(labels, preds)
    
    # Returning a dictionary containing the computed metrics
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }


Define training arguments

In [31]:
training_args = TrainingArguments(
    output_dir='./output', 
    do_train=True,
    do_eval=True,
    num_train_epochs=100,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=16,
    warmup_steps=100,                
    weight_decay=0.05,
    logging_strategy='steps',
    logging_dir='./logs',            
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps", 
    load_best_model_at_end=True
)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer = Trainer(
    model=model,
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=test_dataloader,            
    compute_metrics= compute_metrics
)

In [33]:
%%time
trainer.train()

  0%|          | 0/2200 [00:00<?, ?it/s]

{'loss': 4.8776, 'grad_norm': 5.240514278411865, 'learning_rate': 2.5e-05, 'epoch': 2.27}


  0%|          | 0/15 [00:00<?, ?it/s]

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 4.767058849334717, 'eval_Accuracy': 0.034934497816593885, 'eval_F1': 0.004294139864609663, 'eval_Precision': 0.0026765009523630214, 'eval_Recall': 0.021212121212121213, 'eval_runtime': 1.3475, 'eval_samples_per_second': 169.945, 'eval_steps_per_second': 11.132, 'epoch': 2.27}
{'loss': 4.514, 'grad_norm': 5.23051118850708, 'learning_rate': 5e-05, 'epoch': 4.55}


...

{'eval_loss': 0.8259606957435608, 'eval_Accuracy': 0.851528384279476, 'eval_F1': 0.8036012392448035, 'eval_Precision': 0.8094059405940596, 'eval_Recall': 0.8182696841112683, 'eval_runtime': 0.6932, 'eval_samples_per_second': 330.348, 'eval_steps_per_second': 21.638, 'epoch': 97.73}
{'loss': 0.0609, 'grad_norm': 0.11326982080936432, 'learning_rate': 0.0, 'epoch': 100.0}


  0%|          | 0/15 [00:00<?, ?it/s]

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.8264638781547546, 'eval_Accuracy': 0.851528384279476, 'eval_F1': 0.8027695626705528, 'eval_Precision': 0.808038661008958, 'eval_Recall': 0.8182696841112683, 'eval_runtime': 0.6698, 'eval_samples_per_second': 341.908, 'eval_steps_per_second': 22.396, 'epoch': 100.0}
{'train_runtime': 966.3213, 'train_samples_per_second': 70.887, 'train_steps_per_second': 2.277, 'train_loss': 0.5994519558819857, 'epoch': 100.0}
CPU times: user 13min 34s, sys: 55.9 s, total: 14min 30s
Wall time: 16min 6s


TrainOutput(global_step=2200, training_loss=0.5994519558819857, metrics={'train_runtime': 966.3213, 'train_samples_per_second': 70.887, 'train_steps_per_second': 2.277, 'total_flos': 881077530450000.0, 'train_loss': 0.5994519558819857, 'epoch': 100.0})

Evaluate trainer

In [34]:
q=[trainer.evaluate(eval_dataset=df2) for df2 in [train_dataloader, test_dataloader]]
pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.073348,0.972263,0.981240,0.986588,0.980843
test,0.805604,0.834061,0.788751,0.801438,0.806388


Save model and tokenizer

In [35]:
model_path = "base"

In [36]:
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('base/tokenizer_config.json',
 'base/special_tokens_map.json',
 'base/vocab.txt',
 'base/added_tokens.json')

Load model and tokenizer

In [42]:
new_model = BertForSequenceClassification.from_pretrained(model_path)
new_tokenizer= BertTokenizerFast.from_pretrained(model_path)

In [38]:
pipe= pipeline("sentiment-analysis", model=new_model, tokenizer=new_tokenizer, device='mps')

In [39]:
pipe("Hello")

[{'label': 'greeting', 'score': 0.9863651990890503}]

## Predict

In [43]:
def predict(text):
    # Tokenize the input text and convert it to PyTorch tensors
    inputs = new_tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cpu")

    # Pass the input tensors through the model
    outputs = new_model(**inputs)

    # Softmax the output to get probabilities
    probs = outputs[0].softmax(1)

    # Get the predicted label index
    pred_label_idx = probs.argmax()

    # Convert the predicted label index to the actual label using model configuration
    pred_label = new_model.config.id2label[pred_label_idx.item()]

    return pred_label


In [44]:
text = "Hello"
predict(text)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


'greeting'

In [50]:
def model_response(text):
    response = ''
    score = pipe(text)[0]['score']
    tag = pipe(text)[0]['label']
    if score < 0.8:
        response += "Don't understand"
    else:
        label = new_model.config.label2id[pipe(text)[0]['label']]
        response = random.choice(intents['intents'][label]['responses'])
    return response, tag

In [56]:
model_response('have a gym')

("The college has excellent gym facilities with a large amount of training equipment, 'For more information please visit - 'https://www.mmcoe.edu.in/index.php/campus/gymnasium'",
 'Campus-Gymnasium')